In [1460]:
import pandas as pd
import json
import mysql.connector
from sqlalchemy import create_engine
from sqlalchemy import URL

In [1461]:
pd.options.display.max_columns = 100

In [1462]:
username = 'root'
password='root'
host='localhost'
database='jmdb'

engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}/{database}")


In [1463]:
df = pd.read_csv("emdb.csv",encoding="utf-16", sep = ";")

In [1464]:
df = df.rename(columns = {v : v.strip() for v in df.columns})

In [1465]:
print(df.columns)

Index(['Numéro', 'Titre', 'Titre alternatif', 'Année', 'Pays', 'Genre(s)',
       'Langues', 'Durée', 'Chemin de l'affiche', 'Réalisateur(s)',
       'Scénariste(s)', 'Compositeur(s)', 'Studio', 'Distribution',
       'Bande-annonce', 'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
       'Commentaires', 'Etiquettes', 'Support', 'Taille du fichier',
       'ID TheMovieDb', 'ID IMDb'],
      dtype='object')


# table movie , director et intermediaire

In [1466]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [1467]:
# Modifier le chemin de l'affiche pour ne conserver que la dernière partie
df_movie["Chemin de l'affiche"] = df_movie["Chemin de l'affiche"].astype(str).apply(lambda x: x.split('\\')[-1])

# Afficher le DataFrame résultant
print(df_movie.head())  # Affiche les premières lignes du DataFrame

   Numéro                 Titre                         Titre alternatif  \
0    3063           Underground                Подземље (Titre original)   
1    1068  chat noir chat blanc  Crna macka, beli macor (Titre original)   
2     335   Le temps des Gitans          Dom za vešanje (Titre original)   
3    1718             The Tribe                  Плем'я (Titre original)   
4    2525                  Olga                                      NaN   

  Année  Durée Chemin de l'affiche  \
0  1995    170          003140.jpg   
1  1998    127          001091.jpg   
2  1988    142          000336.jpg   
3  2014    130          001780.jpg   
4  2021     90          002599.jpg   

                                       Bande-annonce  \
0                www.youtube.com/watch?v=rYTvmLvLchs   
1  www.imdb.com/video/vi2662990105?playlistId=tt0...   
2                www.youtube.com/watch?v=1cyBXPMurJY   
3                www.youtube.com/watch?v=2FKxenXcu1I   
4                www.youtube.com/w

In [1468]:
list_director_movie = ["Réalisateur(s)", 'Numéro']
df_director_movie = df.copy()[list_director_movie]

In [1469]:
# df_director_movie ["Réalisateur(s)"] = df_director_movie["Réalisateur(s)"].str.split(",")
df_director_movie["Réalisateur(s)"] = df_director_movie["Réalisateur(s)"].str.split(",\s*")

In [1470]:
df_director_movie  = df_director_movie.explode("Réalisateur(s)")

In [1471]:
df_director  = df_director_movie.explode("Réalisateur(s)").drop_duplicates("Réalisateur(s)").reset_index(drop=True).reset_index()
df_director = df_director.rename(columns={"index":"id"}).drop(columns="Numéro")

#.reset_index().rename(columns={"index":"id"})

In [1472]:
df_intermediaire_movie_director = df_director.merge(df_director_movie, on = "Réalisateur(s)").drop(columns="Réalisateur(s)")

In [1473]:
# Remplacer les valeurs NULL de la colonne 'id' par 0
df_director['id'] = df_director['id'].fillna(0)
# Convertir la colonne 'id' en type entier
df_director['id'] = df_director['id'].astype(int)

# Réorganiser les ID si nécessaire pour éviter les conflits
max_id = df_director['id'].max()
if max_id > 0:
    df_director['id'] = df_director['id'] + 1

In [1474]:
display(df_director)

,id,Réalisateur(s)
0,1,Emir Kusturica
1,2,Myroslav Slaboshpytskyi
2,3,Elie Grappe
3,4,Deniz Gamze Ergüven
4,5,Prachya Pinkaew
...,...,...
1733,1734,Dominic Bachy
1734,1735,Federica Bertelli
1735,1736,Andreï Sokolov
1736,1737,Daniel Costelle


In [1475]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3063,Underground,Подземље (Titre original),1995,170,003140.jpg,www.youtube.com/watch?v=rYTvmLvLchs,Underground,"En avril 1941, des militants communistes serbe...",[Seagate Portable Drive]F:\#DIVX\DIVX MOVIEZ\D...,Inconnu,NaN,Fichier multimédia,2.68 GB,movie/11902,114787.0
1,1068,chat noir chat blanc,"Crna macka, beli macor (Titre original)",1998,127,001091.jpg,www.imdb.com/video/vi2662990105?playlistId=tt0...,NaN,Un petit malfrat est dépassé par les événement...,NaN,Inconnu,NaN,DVD R/RW,NaN,NaN,118843.0
2,335,Le temps des Gitans,Dom za vešanje (Titre original),1988,142,000336.jpg,www.youtube.com/watch?v=1cyBXPMurJY,NaN,"Perhan, fils bâtard d’un soldat slovène et d’u...",NaN,Inconnu,NaN,DVD original,NaN,movie/20123,97223.0
3,1718,The Tribe,Плем'я (Titre original),2014,130,001780.jpg,www.youtube.com/watch?v=2FKxenXcu1I,Love and hate need no translation.,"Sergey, sourd et muet, entre dans un internat ...",[Seagate Portable Drive]G:\#DIVX\DIVX MOVIEZ\D...,Matroska (MKV),NaN,Fichier multimédia,1.44 GB,movie/271397,1745787.0
4,2525,Olga,NaN,2021,90,002599.jpg,www.youtube.com/watch?v=S6xf-JxQTUI,NaN,Le film est présenté à la Semaine Internationa...,NaN,Matroska (MKV),NaN,Fichier multimédia,NaN,movie/749142,13145186.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3499,2431,Les Robins des bois d'à peu près Alexandre Dumas,NaN,1996,105,002497.jpg,NaN,NaN,NaN,[Seagate Portable Drive]G:\#DIVX\DIVX MOVIEZ\C...,AVI (Microsoft),NaN,Fichier multimédia,702 MB,NaN,NaN
3500,844,"Betty Boop (Cartoon parade, vol. 2)",NaN,1936,58,000856.jpg,NaN,NaN,NaN,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3501,843,"Betty Boop (Cartoon parade, vol. 1)",NaN,1936,52,000855.jpg,NaN,NaN,Betty Boop et le petit roi - La mouche - Pas m...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3502,107,Avant-Garde: Experimental cinema of the 1920s...,NaN,1920,0,000107.jpg,NaN,NaN,"In the latter half of the 20th Century, Raymon...",NaN,Inconnu,"Disc 1 Le Retour à la raison (Man Ray, France,...",DVD original,NaN,movie/508159,NaN


In [1476]:
display(df_intermediaire_movie_director)

,id,Numéro
0,0,3063
1,0,1068
2,0,335
3,1,1718
4,2,2525
...,...,...
3771,1735,159
3772,1736,1155
3773,1736,1154
3774,1737,1155


In [1477]:
# Trouver l'index de la ligne avec le 'Réalisateur(s)' NULL
null_director_index = df_director[df_director['Réalisateur(s)'].isnull()].index

# Si une ligne avec 'Réalisateur(s)' NULL est trouvée
if not null_director_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_director.loc[null_director_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_director['id'].max()
    if max_id > 0:
        df_director.loc[df_director.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_director.head())

# Enregistrer le DataFrame modifié dans la table 'director' de la base de données
df_director.to_sql("director", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_director = df_director.merge(df_director_movie, on="Réalisateur(s)").drop(columns="Réalisateur(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_director.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_director.to_sql("movie_director", index=False, con=engine, if_exists="replace")

     id           Réalisateur(s)
0  1739           Emir Kusturica
1     2  Myroslav Slaboshpytskyi
2     3              Elie Grappe
3     4      Deniz Gamze Ergüven
4     5          Prachya Pinkaew
     id  Numéro
0  1739    3063
1  1739    1068
2  1739     335
3     2    1718
4     3    2525


3776

In [1478]:
# df_director.to_sql("director",index=False,con = engine,if_exists = "replace" )

In [1479]:
# df_intermediaire_movie_director.to_sql("movie_director",index=False,con = engine,if_exists = "replace" )

In [1480]:
df_movie.to_sql("movies",index=False,con = engine,if_exists = "replace" )

3504

# table movie, screenwriter et intermediaire

In [1481]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [1482]:
list_screenwriter_movie = ["Scénariste(s)", 'Numéro']
df_screenwriter_movie = df.copy()[list_screenwriter_movie]

In [1483]:
df_screenwriter_movie ["Scénariste(s)"] = df_screenwriter_movie["Scénariste(s)"].str.split(",\s*")

In [1484]:
df_screenwriter_movie  = df_screenwriter_movie.explode("Scénariste(s)")

In [1485]:
df_screenwriter  = df_screenwriter_movie.explode("Scénariste(s)").drop_duplicates("Scénariste(s)").reset_index(drop=True).reset_index()
df_screenwriter = df_screenwriter.rename(columns={"index":"id"}).drop(columns="Numéro")

In [1486]:
df_intermediaire_movie_screenwriter = df_screenwriter.merge(df_screenwriter_movie, on = "Scénariste(s)").drop(columns="Scénariste(s)")

In [1487]:
display(df_screenwriter)

,id,Scénariste(s)
0,0,Emir Kusturica
1,1,Dušan Kovačević
2,2,Gordan Mihic
3,3,Gordan Mihić
4,4,Myroslav Slaboshpytskyi
...,...,...
3800,3800,Georg Büchner
3801,3801,Ulrich Zieger
3802,3802,Anselma Heine
3803,3803,Richard Oswald


In [1488]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3063,Underground,Подземље (Titre original),1995,170,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=rYTvmLvLchs,Underground,"En avril 1941, des militants communistes serbe...",[Seagate Portable Drive]F:\#DIVX\DIVX MOVIEZ\D...,Inconnu,NaN,Fichier multimédia,2.68 GB,movie/11902,114787.0
1,1068,chat noir chat blanc,"Crna macka, beli macor (Titre original)",1998,127,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.imdb.com/video/vi2662990105?playlistId=tt0...,NaN,Un petit malfrat est dépassé par les événement...,NaN,Inconnu,NaN,DVD R/RW,NaN,NaN,118843.0
2,335,Le temps des Gitans,Dom za vešanje (Titre original),1988,142,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=1cyBXPMurJY,NaN,"Perhan, fils bâtard d’un soldat slovène et d’u...",NaN,Inconnu,NaN,DVD original,NaN,movie/20123,97223.0
3,1718,The Tribe,Плем'я (Titre original),2014,130,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=2FKxenXcu1I,Love and hate need no translation.,"Sergey, sourd et muet, entre dans un internat ...",[Seagate Portable Drive]G:\#DIVX\DIVX MOVIEZ\D...,Matroska (MKV),NaN,Fichier multimédia,1.44 GB,movie/271397,1745787.0
4,2525,Olga,NaN,2021,90,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=S6xf-JxQTUI,NaN,Le film est présenté à la Semaine Internationa...,NaN,Matroska (MKV),NaN,Fichier multimédia,NaN,movie/749142,13145186.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3499,2431,Les Robins des bois d'à peu près Alexandre Dumas,NaN,1996,105,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,NaN,[Seagate Portable Drive]G:\#DIVX\DIVX MOVIEZ\C...,AVI (Microsoft),NaN,Fichier multimédia,702 MB,NaN,NaN
3500,844,"Betty Boop (Cartoon parade, vol. 2)",NaN,1936,58,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,NaN,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3501,843,"Betty Boop (Cartoon parade, vol. 1)",NaN,1936,52,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Betty Boop et le petit roi - La mouche - Pas m...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3502,107,Avant-Garde: Experimental cinema of the 1920s...,NaN,1920,0,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,"In the latter half of the 20th Century, Raymon...",NaN,Inconnu,"Disc 1 Le Retour à la raison (Man Ray, France,...",DVD original,NaN,movie/508159,NaN


In [1489]:
display(df_intermediaire_movie_screenwriter)

,id,Numéro
0,0,3063
1,0,335
2,1,3063
3,2,1068
4,3,335
...,...,...
6811,3800,255
6812,3801,3164
6813,3802,1945
6814,3803,1945


In [1490]:
# Trouver l'index de la ligne avec le 'Scénariste(s)' NULL
null_screenwriter_index = df_screenwriter[df_screenwriter['Scénariste(s)'].isnull()].index

# Si une ligne avec 'Scénariste(s)' NULL est trouvée
if not null_screenwriter_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_screenwriter.loc[null_screenwriter_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_screenwriter['id'].max()
    if max_id > 0:
        df_screenwriter.loc[df_screenwriter.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_screenwriter.head())

# Enregistrer le DataFrame modifié dans la table 'screenwriter' de la base de données
df_screenwriter.to_sql("screenwriter", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_screenwriter = df_screenwriter.merge(df_screenwriter_movie, on="Scénariste(s)").drop(columns="Scénariste(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_screenwriter.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_screenwriter.to_sql("movie_screenwriter", index=False, con=engine, if_exists="replace")


     id            Scénariste(s)
0  3805           Emir Kusturica
1     1          Dušan Kovačević
2     2             Gordan Mihic
3     3             Gordan Mihić
4     4  Myroslav Slaboshpytskyi
     id  Numéro
0  3805    3063
1  3805     335
2     1    3063
3     2    1068
4     3     335


6816

In [1491]:
# df_screenwriter.to_sql("screenwriter",index=False,con = engine,if_exists = "replace" )

In [1492]:
# df_intermediaire_movie_screenwriter.to_sql("movie_screenwriter",index=False,con = engine,if_exists = "replace" )

# table movie, music et intermediaire

In [1493]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [1494]:
list_music_movie = ["Compositeur(s)", 'Numéro']
df_music_movie = df.copy()[list_music_movie]

In [1495]:
df_music_movie ["Compositeur(s)"] = df_music_movie["Compositeur(s)"].str.split(",\s*")

In [1496]:
df_music_movie  = df_music_movie.explode("Compositeur(s)")

In [1497]:
df_music  = df_music_movie.explode("Compositeur(s)").drop_duplicates("Compositeur(s)").reset_index(drop=True).reset_index()
df_music = df_music.rename(columns={"index":"id"}).drop(columns="Numéro")

In [1498]:
df_intermediaire_movie_music = df_music.merge(df_music_movie, on = "Compositeur(s)").drop(columns="Compositeur(s)")

In [1499]:
display(df_music)

,id,Compositeur(s)
0,0,Goran Bregović
1,1,Vojislav Aralica
2,2,Nele Karajlic
3,3,Dejan Sparavalo
4,4,NaN
...,...,...
1250,1250,Peter Schirmann
1251,1251,Carlo Savina
1252,1252,Stéphane Moucha
1253,1253,Klaus Schulze


In [1500]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3063,Underground,Подземље (Titre original),1995,170,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=rYTvmLvLchs,Underground,"En avril 1941, des militants communistes serbe...",[Seagate Portable Drive]F:\#DIVX\DIVX MOVIEZ\D...,Inconnu,NaN,Fichier multimédia,2.68 GB,movie/11902,114787.0
1,1068,chat noir chat blanc,"Crna macka, beli macor (Titre original)",1998,127,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.imdb.com/video/vi2662990105?playlistId=tt0...,NaN,Un petit malfrat est dépassé par les événement...,NaN,Inconnu,NaN,DVD R/RW,NaN,NaN,118843.0
2,335,Le temps des Gitans,Dom za vešanje (Titre original),1988,142,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=1cyBXPMurJY,NaN,"Perhan, fils bâtard d’un soldat slovène et d’u...",NaN,Inconnu,NaN,DVD original,NaN,movie/20123,97223.0
3,1718,The Tribe,Плем'я (Titre original),2014,130,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=2FKxenXcu1I,Love and hate need no translation.,"Sergey, sourd et muet, entre dans un internat ...",[Seagate Portable Drive]G:\#DIVX\DIVX MOVIEZ\D...,Matroska (MKV),NaN,Fichier multimédia,1.44 GB,movie/271397,1745787.0
4,2525,Olga,NaN,2021,90,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=S6xf-JxQTUI,NaN,Le film est présenté à la Semaine Internationa...,NaN,Matroska (MKV),NaN,Fichier multimédia,NaN,movie/749142,13145186.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3499,2431,Les Robins des bois d'à peu près Alexandre Dumas,NaN,1996,105,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,NaN,[Seagate Portable Drive]G:\#DIVX\DIVX MOVIEZ\C...,AVI (Microsoft),NaN,Fichier multimédia,702 MB,NaN,NaN
3500,844,"Betty Boop (Cartoon parade, vol. 2)",NaN,1936,58,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,NaN,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3501,843,"Betty Boop (Cartoon parade, vol. 1)",NaN,1936,52,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Betty Boop et le petit roi - La mouche - Pas m...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3502,107,Avant-Garde: Experimental cinema of the 1920s...,NaN,1920,0,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,"In the latter half of the 20th Century, Raymon...",NaN,Inconnu,"Disc 1 Le Retour à la raison (Man Ray, France,...",DVD original,NaN,movie/508159,NaN


In [1501]:
display(df_intermediaire_movie_music)

,id,Numéro
0,0,3063
1,0,335
2,1,1068
3,2,1068
4,3,1068
...,...,...
3884,1250,212
3885,1251,3232
3886,1252,2008
3887,1253,1844


In [1502]:
# # Trouver l'index de la ligne avec le 'Compositeur(s)' NULL
# null_music_index = df_music[df_music['Compositeur(s)'].isnull()].index

# # Si une ligne avec 'Compositeur(s)' NULL est trouvée
# if not null_music_index.empty:
#     # Attribuer à cette ligne l'ID 0
#     df_music.loc[null_music_index, 'id'] = 0

#     # Décaler l'ID de la première ligne (anciennement index 0)
#     max_id = df_music['id'].max()
#     if max_id > 0:
#         df_music.loc[df_music.index[0], 'id'] = max_id + 1

# # Afficher le DataFrame modifié
# print(df_music.head())

# # Enregistrer le DataFrame modifié dans la table 'music' de la base de données
# df_music.to_sql("music", index=False, con=engine, if_exists="replace")

# # Recréer la table intermédiaire avec les données mises à jour
# df_intermediaire_movie_music = df_music.merge(df_music_movie, on="Compositeur(s)").drop(columns="Compositeur(s)")

# # Afficher la table intermédiaire mise à jour
# print(df_intermediaire_movie_music.head())

# # Enregistrer la table intermédiaire mise à jour dans la base de données
# df_intermediaire_movie_music.to_sql("movie_music", index=False, con=engine, if_exists="replace")


In [1503]:
df_music.to_sql("music",index=False,con = engine,if_exists = "replace" )

1255

In [1504]:
df_intermediaire_movie_music.to_sql("movie_music",index=False,con = engine,if_exists = "replace" )

3889

# table movie, studio et intermediaire

In [1505]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [1506]:
list_studio_movie = ["Studio", 'Numéro']
df_studio_movie = df.copy()[list_studio_movie]

In [1507]:
df_studio_movie ["Studio"] = df_studio_movie["Studio"].str.split(",\s*")

In [1508]:
df_studio_movie  = df_studio_movie.explode("Studio")

In [1509]:
df_studio  = df_studio_movie.explode("Studio").drop_duplicates("Studio").reset_index(drop=True).reset_index()
df_studio = df_studio.rename(columns={"index":"id"}).drop(columns="Numéro")

In [1510]:
df_intermediaire_movie_studio = df_studio.merge(df_studio_movie, on = "Studio").drop(columns="Studio")

In [1511]:
display(df_studio)

,id,Studio
0,0,Komuna
1,1,AFMD
2,2,NaN
3,3,Hubert Bals Fund
4,4,Cinéma Defacto
...,...,...
918,918,Gerald Kargl
919,919,Klaus Kinski Productions
920,920,TFM Distribution
921,921,Solaris Film


In [1512]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3063,Underground,Подземље (Titre original),1995,170,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=rYTvmLvLchs,Underground,"En avril 1941, des militants communistes serbe...",[Seagate Portable Drive]F:\#DIVX\DIVX MOVIEZ\D...,Inconnu,NaN,Fichier multimédia,2.68 GB,movie/11902,114787.0
1,1068,chat noir chat blanc,"Crna macka, beli macor (Titre original)",1998,127,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.imdb.com/video/vi2662990105?playlistId=tt0...,NaN,Un petit malfrat est dépassé par les événement...,NaN,Inconnu,NaN,DVD R/RW,NaN,NaN,118843.0
2,335,Le temps des Gitans,Dom za vešanje (Titre original),1988,142,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=1cyBXPMurJY,NaN,"Perhan, fils bâtard d’un soldat slovène et d’u...",NaN,Inconnu,NaN,DVD original,NaN,movie/20123,97223.0
3,1718,The Tribe,Плем'я (Titre original),2014,130,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=2FKxenXcu1I,Love and hate need no translation.,"Sergey, sourd et muet, entre dans un internat ...",[Seagate Portable Drive]G:\#DIVX\DIVX MOVIEZ\D...,Matroska (MKV),NaN,Fichier multimédia,1.44 GB,movie/271397,1745787.0
4,2525,Olga,NaN,2021,90,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=S6xf-JxQTUI,NaN,Le film est présenté à la Semaine Internationa...,NaN,Matroska (MKV),NaN,Fichier multimédia,NaN,movie/749142,13145186.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3499,2431,Les Robins des bois d'à peu près Alexandre Dumas,NaN,1996,105,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,NaN,[Seagate Portable Drive]G:\#DIVX\DIVX MOVIEZ\C...,AVI (Microsoft),NaN,Fichier multimédia,702 MB,NaN,NaN
3500,844,"Betty Boop (Cartoon parade, vol. 2)",NaN,1936,58,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,NaN,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3501,843,"Betty Boop (Cartoon parade, vol. 1)",NaN,1936,52,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Betty Boop et le petit roi - La mouche - Pas m...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3502,107,Avant-Garde: Experimental cinema of the 1920s...,NaN,1920,0,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,"In the latter half of the 20th Century, Raymon...",NaN,Inconnu,"Disc 1 Le Retour à la raison (Man Ray, France,...",DVD original,NaN,movie/508159,NaN


In [1513]:
display(df_intermediaire_movie_studio)

,id,Numéro
0,0,3063
1,1,1068
2,2,335
3,2,1313
4,2,631
...,...,...
3501,918,1844
3502,919,3016
3503,920,2773
3504,921,1841


In [1514]:
# # Trouver l'index de la ligne avec le 'Studio' NULL
# null_studio_index = df_studio[df_studio['Studio'].isnull()].index

# # Si une ligne avec 'Studio' NULL est trouvée
# if not null_studio_index.empty:
#     # Attribuer à cette ligne l'ID 0
#     df_studio.loc[null_studio_index, 'id'] = 0

#     # Décaler l'ID de la première ligne (anciennement index 0)
#     max_id = df_studio['id'].max()
#     if max_id > 0:
#         df_studio.loc[df_studio.index[0], 'id'] = max_id + 1

# # Afficher le DataFrame modifié
# print(df_studio.head())

# # Enregistrer le DataFrame modifié dans la table 'studio' de la base de données
# df_studio.to_sql("studio", index=False, con=engine, if_exists="replace")

# # Recréer la table intermédiaire avec les données mises à jour
# df_intermediaire_movie_studio = df_studio.merge(df_studio_movie, on="Studio").drop(columns="Studio")

# # Afficher la table intermédiaire mise à jour
# print(df_intermediaire_movie_studio.head())

# # Enregistrer la table intermédiaire mise à jour dans la base de données
# df_intermediaire_movie_studio.to_sql("movie_studio", index=False, con=engine, if_exists="replace")


In [1515]:
df_studio.to_sql("studio",index=False,con = engine,if_exists = "replace" )

923

In [1516]:
df_intermediaire_movie_studio.to_sql("movie_studio",index=False,con = engine,if_exists = "replace" )

3506

# table movie, casting & intermediaire

In [1517]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [1518]:
list_casting_movie = ["Distribution", 'Numéro']
df_casting_movie = df.copy()[list_casting_movie]

In [1519]:
df_casting_movie ["Distribution"] = df_casting_movie["Distribution"].str.split(",\s*")

In [1520]:
df_casting_movie  = df_casting_movie.explode("Distribution")

In [1521]:
df_casting  = df_casting_movie.explode("Distribution").drop_duplicates("Distribution").reset_index(drop=True).reset_index()
df_casting = df_casting.rename(columns={"index":"id"}).drop(columns="Numéro")

In [1522]:
df_intermediaire_movie_casting = df_casting.merge(df_casting_movie, on = "Distribution").drop(columns="Distribution")

In [1523]:
display(df_casting)

,id,Distribution
0,0,Miki Manojlović
1,1,Lazar Ristovski
2,2,Mirjana Joković
3,3,Slavko Štimac
4,4,Ernst Stötzner
...,...,...
9630,9630,Willy Fritsch
9631,9631,Gerda Maurus
9632,9632,Klaus Pohl
9633,9633,Fritz Rasp


In [1524]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3063,Underground,Подземље (Titre original),1995,170,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=rYTvmLvLchs,Underground,"En avril 1941, des militants communistes serbe...",[Seagate Portable Drive]F:\#DIVX\DIVX MOVIEZ\D...,Inconnu,NaN,Fichier multimédia,2.68 GB,movie/11902,114787.0
1,1068,chat noir chat blanc,"Crna macka, beli macor (Titre original)",1998,127,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.imdb.com/video/vi2662990105?playlistId=tt0...,NaN,Un petit malfrat est dépassé par les événement...,NaN,Inconnu,NaN,DVD R/RW,NaN,NaN,118843.0
2,335,Le temps des Gitans,Dom za vešanje (Titre original),1988,142,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=1cyBXPMurJY,NaN,"Perhan, fils bâtard d’un soldat slovène et d’u...",NaN,Inconnu,NaN,DVD original,NaN,movie/20123,97223.0
3,1718,The Tribe,Плем'я (Titre original),2014,130,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=2FKxenXcu1I,Love and hate need no translation.,"Sergey, sourd et muet, entre dans un internat ...",[Seagate Portable Drive]G:\#DIVX\DIVX MOVIEZ\D...,Matroska (MKV),NaN,Fichier multimédia,1.44 GB,movie/271397,1745787.0
4,2525,Olga,NaN,2021,90,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=S6xf-JxQTUI,NaN,Le film est présenté à la Semaine Internationa...,NaN,Matroska (MKV),NaN,Fichier multimédia,NaN,movie/749142,13145186.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3499,2431,Les Robins des bois d'à peu près Alexandre Dumas,NaN,1996,105,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,NaN,[Seagate Portable Drive]G:\#DIVX\DIVX MOVIEZ\C...,AVI (Microsoft),NaN,Fichier multimédia,702 MB,NaN,NaN
3500,844,"Betty Boop (Cartoon parade, vol. 2)",NaN,1936,58,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,NaN,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3501,843,"Betty Boop (Cartoon parade, vol. 1)",NaN,1936,52,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Betty Boop et le petit roi - La mouche - Pas m...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3502,107,Avant-Garde: Experimental cinema of the 1920s...,NaN,1920,0,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,"In the latter half of the 20th Century, Raymon...",NaN,Inconnu,"Disc 1 Le Retour à la raison (Man Ray, France,...",DVD original,NaN,movie/508159,NaN


In [1525]:
display(df_intermediaire_movie_casting)

,id,Numéro
0,0,3063
1,1,3063
2,2,3063
3,3,3063
4,4,3063
...,...,...
16613,9630,2415
16614,9631,2415
16615,9632,2415
16616,9633,2415


In [1526]:
# Trouver l'index de la ligne avec la 'Distribution' NULL
null_casting_index = df_casting[df_casting['Distribution'].isnull()].index

# Si une ligne avec 'Distribution' NULL est trouvée
if not null_casting_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_casting.loc[null_casting_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_casting['id'].max()
    if max_id > 0:
        df_casting.loc[df_casting.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_casting.head())

# Enregistrer le DataFrame modifié dans la table 'casting' de la base de données
df_casting.to_sql("casting", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_casting = df_casting.merge(df_casting_movie, on="Distribution").drop(columns="Distribution")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_casting.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_casting.to_sql("movie_casting", index=False, con=engine, if_exists="replace")


     id     Distribution
0  9635  Miki Manojlović
1     1  Lazar Ristovski
2     2  Mirjana Joković
3     3    Slavko Štimac
4     4   Ernst Stötzner
     id  Numéro
0  9635    3063
1     1    3063
2     2    3063
3     3    3063
4     4    3063


16618

In [1527]:
# df_casting.to_sql("casting",index=False,con = engine,if_exists = "replace" )

In [1528]:
# df_intermediaire_movie_casting.to_sql("movie_casting",index=False,con = engine,if_exists = "replace" )

# table movie, country & intermediaire

In [1529]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [1530]:
list_country_movie = ["Pays", 'Numéro']
df_country_movie = df.copy()[list_country_movie]

In [1531]:
df_country_movie ["Pays"] = df_country_movie["Pays"].str.split(",\s*")

In [1532]:
df_country_movie  = df_country_movie.explode("Pays")

In [1533]:
df_country  = df_country_movie.explode("Pays").drop_duplicates("Pays").reset_index(drop=True).reset_index()
df_country = df_country.rename(columns={"index":"id"}).drop(columns="Numéro")

In [1534]:
df_intermediaire_movie_country = df_country.merge(df_country_movie, on = "Pays").drop(columns="Pays")

In [1535]:
display(df_country)

,id,Pays
0,0,Yougoslavie
1,1,UK
2,2,Turquie
3,3,Thaïlande
4,4,Tchad
5,5,Suisse
6,6,Suède
7,7,Russie
8,8,Royaume-Uni
9,9,Pologne


In [1536]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3063,Underground,Подземље (Titre original),1995,170,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=rYTvmLvLchs,Underground,"En avril 1941, des militants communistes serbe...",[Seagate Portable Drive]F:\#DIVX\DIVX MOVIEZ\D...,Inconnu,NaN,Fichier multimédia,2.68 GB,movie/11902,114787.0
1,1068,chat noir chat blanc,"Crna macka, beli macor (Titre original)",1998,127,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.imdb.com/video/vi2662990105?playlistId=tt0...,NaN,Un petit malfrat est dépassé par les événement...,NaN,Inconnu,NaN,DVD R/RW,NaN,NaN,118843.0
2,335,Le temps des Gitans,Dom za vešanje (Titre original),1988,142,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=1cyBXPMurJY,NaN,"Perhan, fils bâtard d’un soldat slovène et d’u...",NaN,Inconnu,NaN,DVD original,NaN,movie/20123,97223.0
3,1718,The Tribe,Плем'я (Titre original),2014,130,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=2FKxenXcu1I,Love and hate need no translation.,"Sergey, sourd et muet, entre dans un internat ...",[Seagate Portable Drive]G:\#DIVX\DIVX MOVIEZ\D...,Matroska (MKV),NaN,Fichier multimédia,1.44 GB,movie/271397,1745787.0
4,2525,Olga,NaN,2021,90,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=S6xf-JxQTUI,NaN,Le film est présenté à la Semaine Internationa...,NaN,Matroska (MKV),NaN,Fichier multimédia,NaN,movie/749142,13145186.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3499,2431,Les Robins des bois d'à peu près Alexandre Dumas,NaN,1996,105,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,NaN,[Seagate Portable Drive]G:\#DIVX\DIVX MOVIEZ\C...,AVI (Microsoft),NaN,Fichier multimédia,702 MB,NaN,NaN
3500,844,"Betty Boop (Cartoon parade, vol. 2)",NaN,1936,58,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,NaN,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3501,843,"Betty Boop (Cartoon parade, vol. 1)",NaN,1936,52,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Betty Boop et le petit roi - La mouche - Pas m...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3502,107,Avant-Garde: Experimental cinema of the 1920s...,NaN,1920,0,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,"In the latter half of the 20th Century, Raymon...",NaN,Inconnu,"Disc 1 Le Retour à la raison (Man Ray, France,...",DVD original,NaN,movie/508159,NaN


In [1537]:
display(df_intermediaire_movie_country)

,id,Numéro
0,0,3063
1,0,1068
2,0,335
3,1,1718
4,1,2525
...,...,...
3499,44,2431
3500,44,844
3501,44,843
3502,44,107


In [1538]:
# Trouver l'index de la ligne avec le 'Pays' NULL
null_country_index = df_country[df_country['Pays'].isnull()].index

# Si une ligne avec 'Pays' NULL est trouvée
if not null_country_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_country.loc[null_country_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_country['id'].max()
    if max_id > 0:
        df_country.loc[df_country.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_country.head())

# Enregistrer le DataFrame modifié dans la table 'country' de la base de données
df_country.to_sql("country", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_country = df_country.merge(df_country_movie, on="Pays").drop(columns="Pays")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_country.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_country.to_sql("movie_country", index=False, con=engine, if_exists="replace")

   id         Pays
0  44  Yougoslavie
1   1           UK
2   2      Turquie
3   3    Thaïlande
4   4        Tchad
   id  Numéro
0  44    3063
1  44    1068
2  44     335
3   1    1718
4   1    2525


3504

In [1539]:
df_country.to_sql("country",index=False,con = engine,if_exists = "replace" )

45

In [1540]:
df_intermediaire_movie_country.to_sql("movie_country",index=False,con = engine,if_exists = "replace" )

3504

# table movie, genre & intermediaire

In [1541]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [1542]:
list_genre_movie = ["Genre(s)", 'Numéro']
df_genre_movie = df.copy()[list_genre_movie]

In [1543]:
df_genre_movie ["Genre(s)"] = df_genre_movie["Genre(s)"].str.split(",\s*")

In [1544]:
df_genre_movie  = df_genre_movie.explode("Genre(s)")

In [1545]:
df_genre  = df_genre_movie.explode("Genre(s)").drop_duplicates("Genre(s)").reset_index(drop=True).reset_index()
df_genre = df_genre.rename(columns={"index":"id"}).drop(columns="Numéro")

In [1546]:
df_intermediaire_movie_genre = df_genre.merge(df_genre_movie, on = "Genre(s)").drop(columns="Genre(s)")

In [1547]:
display(df_genre)

,id,Genre(s)
0,0,Comédie
1,1,Drame
2,2,Guerre
3,3,Fantastique
4,4,Policier
5,5,Sport
6,6,Action
7,7,Asia
8,8,Romantique
9,9,Animation


In [1548]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3063,Underground,Подземље (Titre original),1995,170,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=rYTvmLvLchs,Underground,"En avril 1941, des militants communistes serbe...",[Seagate Portable Drive]F:\#DIVX\DIVX MOVIEZ\D...,Inconnu,NaN,Fichier multimédia,2.68 GB,movie/11902,114787.0
1,1068,chat noir chat blanc,"Crna macka, beli macor (Titre original)",1998,127,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.imdb.com/video/vi2662990105?playlistId=tt0...,NaN,Un petit malfrat est dépassé par les événement...,NaN,Inconnu,NaN,DVD R/RW,NaN,NaN,118843.0
2,335,Le temps des Gitans,Dom za vešanje (Titre original),1988,142,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=1cyBXPMurJY,NaN,"Perhan, fils bâtard d’un soldat slovène et d’u...",NaN,Inconnu,NaN,DVD original,NaN,movie/20123,97223.0
3,1718,The Tribe,Плем'я (Titre original),2014,130,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=2FKxenXcu1I,Love and hate need no translation.,"Sergey, sourd et muet, entre dans un internat ...",[Seagate Portable Drive]G:\#DIVX\DIVX MOVIEZ\D...,Matroska (MKV),NaN,Fichier multimédia,1.44 GB,movie/271397,1745787.0
4,2525,Olga,NaN,2021,90,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=S6xf-JxQTUI,NaN,Le film est présenté à la Semaine Internationa...,NaN,Matroska (MKV),NaN,Fichier multimédia,NaN,movie/749142,13145186.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3499,2431,Les Robins des bois d'à peu près Alexandre Dumas,NaN,1996,105,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,NaN,[Seagate Portable Drive]G:\#DIVX\DIVX MOVIEZ\C...,AVI (Microsoft),NaN,Fichier multimédia,702 MB,NaN,NaN
3500,844,"Betty Boop (Cartoon parade, vol. 2)",NaN,1936,58,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,NaN,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3501,843,"Betty Boop (Cartoon parade, vol. 1)",NaN,1936,52,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Betty Boop et le petit roi - La mouche - Pas m...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3502,107,Avant-Garde: Experimental cinema of the 1920s...,NaN,1920,0,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,"In the latter half of the 20th Century, Raymon...",NaN,Inconnu,"Disc 1 Le Retour à la raison (Man Ray, France,...",DVD original,NaN,movie/508159,NaN


In [1549]:
display(df_intermediaire_movie_genre)

,id,Numéro
0,0,3063
1,0,1068
2,0,1465
3,0,2266
4,0,2945
...,...,...
7863,37,2148
7864,37,2431
7865,38,75
7866,38,3113


In [1550]:
# Trouver l'index de la ligne avec le 'Genre(s)' NULL
null_genre_index = df_genre[df_genre['Genre(s)'].isnull()].index

# Si une ligne avec 'Genre(s)' NULL est trouvée
if not null_genre_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_genre.loc[null_genre_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_genre['id'].max()
    if max_id > 0:
        df_genre.loc[df_genre.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_genre.head())

# Enregistrer le DataFrame modifié dans la table 'genre' de la base de données
df_genre.to_sql("genre", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_genre = df_genre.merge(df_genre_movie, on="Genre(s)").drop(columns="Genre(s)")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_genre.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_genre.to_sql("movie_genre", index=False, con=engine, if_exists="replace")


   id     Genre(s)
0  38      Comédie
1   1        Drame
2   2       Guerre
3   3  Fantastique
4   4     Policier
   id  Numéro
0  38    3063
1  38    1068
2  38    1465
3  38    2266
4  38    2945


7868

In [1551]:
df_genre.to_sql("genre",index=False,con = engine,if_exists = "replace" )

39

In [1552]:
df_intermediaire_movie_genre.to_sql("movie_genre",index=False,con = engine,if_exists = "replace" )

7868

# table movie, language & intermediaire

In [1553]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [1554]:
list_language_movie = ["Langues", 'Numéro']
df_language_movie = df.copy()[list_language_movie]

In [1555]:
df_language_movie ["Langues"] = df_language_movie["Langues"].str.split(",\s*")

In [1556]:
df_language_movie  = df_language_movie.explode("Langues")

In [1557]:
df_language  = df_language_movie.explode("Langues").drop_duplicates("Langues").reset_index(drop=True).reset_index()
df_language = df_language.rename(columns={"index":"id"}).drop(columns="Numéro")

In [1558]:
df_intermediaire_movie_language = df_language.merge(df_language_movie, on = "Langues").drop(columns="Langues")

In [1559]:
display(df_language)

,id,Langues
0,0,NaN
1,1,Serbe
2,2,Bulgare
3,3,Français
4,4,Russe
5,5,Ukrainien
6,6,Turc
7,7,Anglais
8,8,Thaïlandais
9,9,Arabe


In [1560]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3063,Underground,Подземље (Titre original),1995,170,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=rYTvmLvLchs,Underground,"En avril 1941, des militants communistes serbe...",[Seagate Portable Drive]F:\#DIVX\DIVX MOVIEZ\D...,Inconnu,NaN,Fichier multimédia,2.68 GB,movie/11902,114787.0
1,1068,chat noir chat blanc,"Crna macka, beli macor (Titre original)",1998,127,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.imdb.com/video/vi2662990105?playlistId=tt0...,NaN,Un petit malfrat est dépassé par les événement...,NaN,Inconnu,NaN,DVD R/RW,NaN,NaN,118843.0
2,335,Le temps des Gitans,Dom za vešanje (Titre original),1988,142,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=1cyBXPMurJY,NaN,"Perhan, fils bâtard d’un soldat slovène et d’u...",NaN,Inconnu,NaN,DVD original,NaN,movie/20123,97223.0
3,1718,The Tribe,Плем'я (Titre original),2014,130,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=2FKxenXcu1I,Love and hate need no translation.,"Sergey, sourd et muet, entre dans un internat ...",[Seagate Portable Drive]G:\#DIVX\DIVX MOVIEZ\D...,Matroska (MKV),NaN,Fichier multimédia,1.44 GB,movie/271397,1745787.0
4,2525,Olga,NaN,2021,90,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=S6xf-JxQTUI,NaN,Le film est présenté à la Semaine Internationa...,NaN,Matroska (MKV),NaN,Fichier multimédia,NaN,movie/749142,13145186.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3499,2431,Les Robins des bois d'à peu près Alexandre Dumas,NaN,1996,105,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,NaN,[Seagate Portable Drive]G:\#DIVX\DIVX MOVIEZ\C...,AVI (Microsoft),NaN,Fichier multimédia,702 MB,NaN,NaN
3500,844,"Betty Boop (Cartoon parade, vol. 2)",NaN,1936,58,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,NaN,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3501,843,"Betty Boop (Cartoon parade, vol. 1)",NaN,1936,52,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Betty Boop et le petit roi - La mouche - Pas m...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3502,107,Avant-Garde: Experimental cinema of the 1920s...,NaN,1920,0,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,"In the latter half of the 20th Century, Raymon...",NaN,Inconnu,"Disc 1 Le Retour à la raison (Man Ray, France,...",DVD original,NaN,movie/508159,NaN


In [1561]:
display(df_intermediaire_movie_language)

,id,Numéro
0,0,3063
1,0,335
2,0,1718
3,0,1111
4,0,1414
...,...,...
4937,43,3474
4938,43,3473
4939,44,926
4940,44,1053


In [1562]:
# # Trouver l'index de la ligne avec les 'Langues' NULL
# null_language_index = df_language[df_language['Langues'].isnull()].index

# # Si une ligne avec 'Langues' NULL est trouvée
# if not null_language_index.empty:
#     # Attribuer à cette ligne l'ID 0
#     df_language.loc[null_language_index, 'id'] = 0

#     # Décaler l'ID de la première ligne (anciennement index 0)
#     max_id = df_language['id'].max()
#     if max_id > 0:
#         df_language.loc[df_language.index[0], 'id'] = max_id + 1

# # Afficher le DataFrame modifié
# print(df_language.head())

# # Enregistrer le DataFrame modifié dans la table 'language' de la base de données
# df_language.to_sql("language", index=False, con=engine, if_exists="replace")

# # Recréer la table intermédiaire avec les données mises à jour
# df_intermediaire_movie_language = df_language.merge(df_language_movie, on="Langues").drop(columns="Langues")

# # Afficher la table intermédiaire mise à jour
# print(df_intermediaire_movie_language.head())

# # Enregistrer la table intermédiaire mise à jour dans la base de données
# df_intermediaire_movie_language.to_sql("movie_language", index=False, con=engine, if_exists="replace")


In [1563]:
df_language.to_sql("language",index=False,con = engine,if_exists = "replace" )

45

In [1564]:
df_intermediaire_movie_language.to_sql("movie_language",index=False,con = engine,if_exists = "replace" )

4942

# table, movie, tag & intermediaire

In [1565]:
list_col_movies = ['Numéro', "Titre" , 'Titre alternatif',
                   'Année','Durée', "Chemin de l'affiche",'Bande-annonce', 
                   'Synopsis', 'Scénario', 'Emplacement', 'Conteneur',
                   'Commentaires', 'Support', 'Taille du fichier','ID TheMovieDb', 'ID IMDb'] #rajouter le caracteristiques de films
df_movie = df.copy()[list_col_movies]

In [1566]:
list_tag_movie = ["Etiquettes", 'Numéro']
df_tag_movie = df.copy()[list_tag_movie]

In [1567]:
df_tag_movie ["Etiquettes"] = df_tag_movie["Etiquettes"].str.split(",\s*")

In [1568]:
df_tag_movie  = df_tag_movie.explode("Etiquettes")

In [1569]:
df_tag = df_tag_movie.explode("Etiquettes").drop_duplicates("Etiquettes").reset_index(drop=True).reset_index()
df_tag = df_tag.rename(columns={"index":"id"}).drop(columns="Numéro")

In [1570]:
df_intermediaire_movie_tag = df_tag.merge(df_tag_movie, on = "Etiquettes").drop(columns="Etiquettes")

In [1571]:
display(df_tag)

,id,Etiquettes
0,0,Palme d'Or
1,1,NaN
2,2,Ados
3,3,Ecole
4,4,Sport
...,...,...
200,200,Party
201,201,Ripoux
202,202,Watergate
203,203,Hotel


In [1572]:
display(df_movie)

,Numéro,Titre,Titre alternatif,Année,Durée,Chemin de l'affiche,Bande-annonce,Synopsis,Scénario,Emplacement,Conteneur,Commentaires,Support,Taille du fichier,ID TheMovieDb,ID IMDb
0,3063,Underground,Подземље (Titre original),1995,170,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=rYTvmLvLchs,Underground,"En avril 1941, des militants communistes serbe...",[Seagate Portable Drive]F:\#DIVX\DIVX MOVIEZ\D...,Inconnu,NaN,Fichier multimédia,2.68 GB,movie/11902,114787.0
1,1068,chat noir chat blanc,"Crna macka, beli macor (Titre original)",1998,127,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.imdb.com/video/vi2662990105?playlistId=tt0...,NaN,Un petit malfrat est dépassé par les événement...,NaN,Inconnu,NaN,DVD R/RW,NaN,NaN,118843.0
2,335,Le temps des Gitans,Dom za vešanje (Titre original),1988,142,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=1cyBXPMurJY,NaN,"Perhan, fils bâtard d’un soldat slovène et d’u...",NaN,Inconnu,NaN,DVD original,NaN,movie/20123,97223.0
3,1718,The Tribe,Плем'я (Titre original),2014,130,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=2FKxenXcu1I,Love and hate need no translation.,"Sergey, sourd et muet, entre dans un internat ...",[Seagate Portable Drive]G:\#DIVX\DIVX MOVIEZ\D...,Matroska (MKV),NaN,Fichier multimédia,1.44 GB,movie/271397,1745787.0
4,2525,Olga,NaN,2021,90,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,www.youtube.com/watch?v=S6xf-JxQTUI,NaN,Le film est présenté à la Semaine Internationa...,NaN,Matroska (MKV),NaN,Fichier multimédia,NaN,movie/749142,13145186.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3499,2431,Les Robins des bois d'à peu près Alexandre Dumas,NaN,1996,105,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,NaN,[Seagate Portable Drive]G:\#DIVX\DIVX MOVIEZ\C...,AVI (Microsoft),NaN,Fichier multimédia,702 MB,NaN,NaN
3500,844,"Betty Boop (Cartoon parade, vol. 2)",NaN,1936,58,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,NaN,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3501,843,"Betty Boop (Cartoon parade, vol. 1)",NaN,1936,52,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,Betty Boop et le petit roi - La mouche - Pas m...,NaN,Inconnu,NaN,DVD original,NaN,NaN,NaN
3502,107,Avant-Garde: Experimental cinema of the 1920s...,NaN,1920,0,C:\Users\BigJulius\AppData\Roaming\EMDB\Covers...,NaN,NaN,"In the latter half of the 20th Century, Raymon...",NaN,Inconnu,"Disc 1 Le Retour à la raison (Man Ray, France,...",DVD original,NaN,movie/508159,NaN


In [1573]:
display(df_intermediaire_movie_tag)

,id,Numéro
0,0,3063
1,0,530
2,0,2698
3,0,203
4,0,2034
...,...,...
4448,204,100
4449,204,101
4450,204,102
4451,204,212


In [1574]:
# Trouver l'index de la ligne avec les 'Etiquettes' NULL
null_tag_index = df_tag[df_tag['Etiquettes'].isnull()].index

# Si une ligne avec 'Etiquettes' NULL est trouvée
if not null_tag_index.empty:
    # Attribuer à cette ligne l'ID 0
    df_tag.loc[null_tag_index, 'id'] = 0

    # Décaler l'ID de la première ligne (anciennement index 0)
    max_id = df_tag['id'].max()
    if max_id > 0:
        df_tag.loc[df_tag.index[0], 'id'] = max_id + 1

# Afficher le DataFrame modifié
print(df_tag.head())

# Enregistrer le DataFrame modifié dans la table 'tag' de la base de données
df_tag.to_sql("tag", index=False, con=engine, if_exists="replace")

# Recréer la table intermédiaire avec les données mises à jour
df_intermediaire_movie_tag = df_tag.merge(df_tag_movie, on="Etiquettes").drop(columns="Etiquettes")

# Afficher la table intermédiaire mise à jour
print(df_intermediaire_movie_tag.head())

# Enregistrer la table intermédiaire mise à jour dans la base de données
df_intermediaire_movie_tag.to_sql("movie_tag", index=False, con=engine, if_exists="replace")


    id  Etiquettes
0  205  Palme d'Or
1    0         NaN
2    2        Ados
3    3       Ecole
4    4       Sport
    id  Numéro
0  205    3063
1  205     530
2  205    2698
3  205     203
4  205    2034


4453

In [1575]:
# df_tag.to_sql("tag",index=False,con = engine,if_exists = "replace" )

In [1576]:
# df_intermediaire_movie_tag.to_sql("movie_tag",index=False,con = engine,if_exists = "replace" )